In [44]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, load_tools
from langchain.tools import Tool,tool,StructuredTool
from langchain.prompts import PromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition,InjectedState
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
    ToolMessage,
    RemoveMessage
)
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools.base import InjectedToolCallId


#structuring
import ast
from langchain_core.output_parsers import JsonOutputParser
#error handling with output parser
from langchain.output_parsers import RetryOutputParser

from dataclasses import dataclass
from typing_extensions import TypedDict
from typing import Annotated, Literal
from pydantic import BaseModel, Field
#get graph visuals
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles
import os
from dotenv import load_dotenv 
load_dotenv()
GOOGLE_API_KEY=os.getenv('google_api_key')

In [45]:
GEMINI_MODEL='gemini-2.0-flash'
llm = ChatGoogleGenerativeAI(google_api_key=GOOGLE_API_KEY, model=GEMINI_MODEL, temperature=0.3)

In [53]:
class State(TypedDict):
    """
    A dictionnary representing the state of the agent.
    """
    messages: Annotated[list, add_messages]
    inbox: dict
    current_draft: dict
    drafts: dict

In [ ]:
import pandas as pd
import os.path
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from email.message import EmailMessage

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://mail.google.com/"]


def main():
  """Shows basic usage of the Gmail API.
  Lists the user's Gmail labels.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    # Call the Gmail API
    service = build("gmail", "v1", credentials=creds)
    results = service.users().labels().list(userId="me").execute()
    labels = results.get("labels", [])

    if not labels:
      print("No labels found.")
      return
    print("Labels:")
    for label in labels:
      print(label["name"])

  except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")


if __name__ == "__main__":
  main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=705846470233-jjuk8on68a7cg9gmhl81r6ftg6mgj44c.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A62807%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=NKvAuGFfrddTck5Jbgj8HS5GXGzKHh&access_type=offline
Labels:
CHAT
SENT
INBOX
IMPORTANT
TRASH
DRAFT
SPAM
CATEGORY_FORUMS
CATEGORY_UPDATES
CATEGORY_PERSONAL
CATEGORY_PROMOTIONS
CATEGORY_SOCIAL
STARRED
UNREAD


In [150]:
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


def show_chatty_threads():
  """Display threads with long conversations(>= 3 messages)
  Return: None

  Load pre-authorized user credentials from the environment.
  TODO(developer) - See https://developers.google.com/identity
  for guides on implementing OAuth2 for the application.
  """
#   creds, _ = google.auth.default()
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  try:
    # create gmail api client
    service = build("gmail", "v1", credentials=creds)

    # pylint: disable=maybe-no-member
    # pylint: disable:R1710
    threads = (
        service.users().threads().list(userId="me").execute().get("threads", [])
    )
    for thread in threads:
      tdata = (
          service.users().threads().get(userId="me", id=thread["id"]).execute()
      )
      nmsgs = len(tdata["messages"])

      # skip if <3 msgs in thread
      if nmsgs > 2:
        msg = tdata["messages"][0]["payload"]
        subject = ""
        for header in msg["headers"]:
          if header["name"] == "Subject":
            subject = header["value"]
            break
        if subject:  # skip if no Subject line
          print(f"- {subject}, {nmsgs}")
    return threads

  except HttpError as error:
    print(f"An error occurred: {error}")


if __name__ == "__main__":
  show_chatty_threads()

- Quick follow up, 5
- Resume Final Version, 3
- Resume Draft #3, 4


In [4]:
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
try:
    # create gmail api client
    service = build("gmail", "v1", credentials=creds)

except HttpError as error:
    print(f"An error occurred: {error}")

In [82]:
@tool
def get_new_mail(tool_call_id: Annotated[str, InjectedToolCallId])-> Command:

    """
    Tool to get the latest emails
    args: none
    return: a dict structured as such {email_id:{email content}}
    """
    ids=service.users().messages().list(userId='me', includeSpamTrash=False ).execute().get('messages',[])
    messages={}
    errors=[]
    for id in ids:
        try:
            mdata=service.users().messages().get(userId="me", id=id["id"], format='full' ).execute()
            id=mdata['id']
            thread=mdata['threadId']
            label=mdata['labelIds']
            headers={h['name']:h['value'] for h in mdata['payload']['headers']}
            sender=headers['From']
            date=headers['Date']
            receiver=headers['To']
            subject=headers['Subject']
            snippet=mdata['snippet']
            body=base64.urlsafe_b64decode(mdata['payload']['parts'][0]['body']['data'].encode("utf-8")).decode("utf-8")
        except:
            try:
                id=mdata['id']
                thread=mdata['threadId']
                label=mdata['labelIds']
                headers={h['name']:h['value'] for h in mdata['payload']['headers']}
                sender=headers['From']
                date=headers['Date']
                receiver=headers['To']
                subject=headers['Subject']
                snippet=mdata['snippet']
                body=base64.urlsafe_b64decode(mdata['payload']['parts'][0]['parts'][0]['body']['data'].encode("utf-8")).decode("utf-8")
            except:
                try: 
                    id=mdata['id']
                    thread=mdata['threadId']
                    label=mdata['labelIds']
                    headers={h['name']:h['value'] for h in mdata['payload']['headers']}
                    sender=headers['From']
                    date=headers['Date']
                    receiver=headers['To']
                    subject=headers['Subject']
                    snippet=mdata['snippet']
                    body=base64.urlsafe_b64decode(mdata['payload']['body']['data'].encode("utf-8")).decode("utf-8")
                except:
                    errors.append(mdata)
        messages[id]={'From':sender,
                    'To':receiver,
                    'Date':date,
                    'label':label,
                    'subject':subject,
                    'Snippet':snippet,
                    'email_id':id,
                    'thread':thread,
                    'body':body
                    }  
    return Command(update={'inbox':messages,
                'messages': [ToolMessage(f'Successfully collected the mail',tool_call_id=tool_call_id)]})

In [94]:
@tool
def create_email(receiver: str, content: str, subject:str,tool_call_id: Annotated[str, InjectedToolCallId]):
    """
    Tool to create a draft of an email
    to create an email you have to make a draft using this tool
    agrs: receiver - the email adress of the person to send the email to
          content - the body of the email
          subject - the subject of the email
    """
    message = EmailMessage()

    message.set_content(content)

    message["To"] = receiver
    message["From"] = "gduser2@workspacesamples.dev"
    message["Subject"] = subject

    # encoded message
    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

    create_message = {"message": {"raw": encoded_message}}
    # pylint: disable=E1101
    draft = (
        service.users()
        .drafts()
        .create(userId="me", body=create_message)
        .execute()
    )


   
    return Command(update={'current_draft':draft,
                'messages': [ToolMessage(f'successfully created a draft ',tool_call_id=tool_call_id)]})

In [95]:
@tool
def show_current_draft(state: Annotated[dict, InjectedState]):
    """
    tool to get the info and show the current_draft
    args: none
    returns: the current draft
    """
    draft=state['current_draft']
    current_draft={}
    d=service.users().drafts().get(userId="me", id=draft['id'] , format='full' ).execute()
    current_draft['id']=d['id']
    current_draft['snippet']=d['message']['snippet']
    headers={h['name']:h['value'] for h in d['message']['payload']['headers']}
    current_draft['To']=headers['To']
    try:
        current_draft['body']=base64.urlsafe_b64decode(d['message']['payload']['parts'][0]['body']['data'].encode("utf-8")).decode("utf-8")
    except:
        try:
            current_draft['body']=base64.urlsafe_b64decode(d['payload']['parts'][0]['parts'][0]['body']['data'].encode("utf-8")).decode("utf-8")
        except:
            current_draft['body']=base64.urlsafe_b64decode(d['payload']['body']['data'].encode("utf-8")).decode("utf-8")

    return current_draft

In [96]:
@tool
def show_inbox(state: Annotated[dict, InjectedState]):
    """
    tool to get the info and show the emails in the inbox
    args: none
    returns: the emails in the inbox
    """
    return state['inbox']

In [97]:
@tool
def display_email(id: str, state: Annotated[dict, InjectedState]):

    """
    tool to display a specific email
    args: id - the id associated with the email to read
    returns: the body of the email
    """
    if id in state['inbox']:
        email=state['inbox'].get(str(id))
        return email['body']


In [ ]:
# @tool
# def get_drafts():

#     """tool to get all the drafts"""
#     draft=service.users().drafts().list(userId='me', includeSpamTrash=False ).execute()
#     drafts=[]
#     for id in draft['drafts']:
#         try:
#             d=service.users().drafts().get(userId="me", id=id['id'] , format='full' ).execute()
#             id=d['id']
#             snippet=d['message']['snippet']
#             headers={h['name']:h['value'] for h in d['message']['payload']['headers']}
#             receiver=headers['To']
#             body=base64.urlsafe_b64decode(d['message']['payload']['parts'][0]['body']['data'].encode("utf-8")).decode("utf-8")
#             drafts.append({
#                             'To':receiver,
#                             'Snippet':snippet,
#                             'id':id,
#                             'body':body
#                             })
#         except:
#             drafts.append(d)
#     return drafts


In [98]:
class gmail_agent:
    def __init__(self):
        self.agent=self._setup()
    def _setup(self):
        langgraph_tools=[get_new_mail,create_email,display_email,show_inbox,show_current_draft]



        graph_builder = StateGraph(State)

        # Modification: tell the LLM which tools it can call
        llm_with_tools = llm.bind_tools(langgraph_tools)
        tool_node = ToolNode(tools=langgraph_tools)
        def chatbot(state: State):
            """ travel assistant that answers user questions about their trip.
            Depending on the request, leverage which tools to use if necessary."""
            return {"messages": [llm_with_tools.invoke(state['messages'])]}

        graph_builder.add_node("chatbot", chatbot)


        graph_builder.add_node("tools", tool_node)
        # Any time a tool is called, we return to the chatbot to decide the next step
        graph_builder.set_entry_point("chatbot")
        graph_builder.add_edge("tools", "chatbot")
        graph_builder.add_conditional_edges(
            "chatbot",
            tools_condition,
        )
        memory=MemorySaver()
        graph=graph_builder.compile(checkpointer=memory)
        return graph
        

    def display_graph(self):
        return display(
                        Image(
                                self.agent.get_graph().draw_mermaid_png(
                                    draw_method=MermaidDrawMethod.API,
                                )
                            )
                        )
    def stream(self,input:str):
        config = {"configurable": {"thread_id": "1"}}
        input_message = HumanMessage(content=input)
        for event in self.agent.stream({"messages": [input_message]}, config, stream_mode="values"):
            event["messages"][-1].pretty_print()

    def chat(self,input:str):
        config = {"configurable": {"thread_id": "1"}}
        response=self.agent.invoke({'messages':HumanMessage(content=str(input))},config)
        return response['messages'][-1].content
    
    def get_state(self, state_val:str):
        config = {"configurable": {"thread_id": "1"}}
        return self.agent.get_state(config).values[state_val]

In [99]:
agent=gmail_agent()

In [93]:
agent.stream('Create a draft of a reply saying thank you and that I have come accross  those job applications where you re required to upload your resume and enter the same information in the fields provided and that it makes sense')

================================ Human Message =================================

Create a draft of a reply saying thank you and that I have come accross  those job applications where you re required to upload your resume and enter the same information in the fields provided and that it makes sense
================================== Ai Message ==================================
Tool Calls:
  create_email (caacf19f-2f67-400a-beb8-15f43bf9cf88)
 Call ID: caacf19f-2f67-400a-beb8-15f43bf9cf88
  Args:
    receiver: giselleb@resumewritingbbc.com
    content: Hi Giselle,

Thank you so much for the cover letter and final versions of the resume! I have definitely come across those job applications where you're required to upload the resume and then re-enter all the same information into separate fields. The plain text version will be a huge help with those, that makes perfect sense.

Thanks again,
Tristan
    subject: Re: Resume Final Version
================================= Tool Message =====

In [92]:
agent.get_state('messages')

[HumanMessage(content='can you get the new mail', additional_kwargs={}, response_metadata={}, id='72534426-9dad-4dbd-b9b4-e3f825c6aa0a'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_new_mail', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-ddcff501-da6b-4a1c-838a-d5a2decf2995-0', tool_calls=[{'name': 'get_new_mail', 'args': {}, 'id': 'a55aea8c-6c60-46bd-9485-99c6f69bd551', 'type': 'tool_call'}], usage_metadata={'input_tokens': 188, 'output_tokens': 5, 'total_tokens': 193, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='Successfully collected the mail', name='get_new_mail', id='a5cb398c-627d-46ce-afe4-5c56ea7ccb4d', tool_call_id='a55aea8c-6c60-46bd-9485-99c6f69bd551'),
 AIMessage(content='OK. I have successfully collected the new mail.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings